In [1]:
import pandas as pd
from datetime import datetime, timedelta
version_in = 'V0_1'
version_out = 'V1'
path_gen = "/home/pol/Escritorio/INVERBIS/2024/CancerColon/"
path_1 = path_gen + "Data/processed_data/CancerColon_semi"+str(version_out)+".csv"
colon = pd.read_csv(path_1, sep= ';')

In [3]:
print(colon.shape)
colon.columns

(22835, 44)


Index(['NASI seudonimizado', 'Subg. químico terapéutico ATC disp',
       'Principio activo ATC disp', 'Prezo PVP nomen disp', 'FECHA',
       'FECHA_FIN', 'Actividad', 'Hospital', 'Motivo de asistencia',
       'Motivo saída', 'PAC', 'Lugar asistencia', 'Motivo asistencia', 'Acto',
       'Prioridade', 'GNA', 'Actividade CAPNOR', 'Código Prestación',
       'Prestación', 'Tipo vía rápida', 'Detalle tipo actividade',
       'Tipo programación', 'Cola', 'Tipo intervención',
       'Orixe PM/Diagnóstico', 'PM/Diagnóstico', 'Categoría CIE',
       'Subcategoría CIE', 'CENTRO', 'SERVICIO', 'MEDICO', 'MUESTRA',
       'DESCRIPCION', 'DATA EXTRACCIÓN', 'Defuncion', 'VDR', 'Colonoscopia',
       'Consultas Digestivo/Medicina interna',
       'Consultas Cirugía General/Oncología', 'Realización TC abdominales',
       'Duracion Lista Espera', 'Medicamento importante',
       'Nº de compras farmaceuticas', 'Coste total medicamentos'],
      dtype='object')

In [4]:
colon['Actividad'].unique().tolist()

['Cita en Medicina interna',
 'Biopsia diagnóstica',
 'Ingreso hospitalario',
 'Cirugia',
 'Alta Hospitalizacion',
 'Visita médico familia',
 'Cita en Oncología',
 'Cita en Cirugía General',
 'Defunción',
 'Entrada Lista de Espera',
 'Salida Lista de Espera',
 'Quimioterapia',
 'Cita en Digestivo',
 'Dispensación medicamento ATC',
 'Asistencia a punto de Atención Continuada',
 'Urgencias Hospitalarias']

In [5]:
colon_tolon = colon[colon['Actividad'] == 'Cirugia']
colon_tolon['GNA'].unique().tolist()

['CIRURXIA XERAL E DIXESTIVA']

In [6]:
colon['GNA'].unique().tolist()

['MEDICINA INTERNA',
 nan,
 'CIRURXIA XERAL E DIXESTIVA',
 'ONCOLOXIA MEDICA',
 'DIXESTIVO']

In [ ]:
def surgery_activities(df, attrib_name):
    traces = []
    grouper_3 = df.groupby('NASI seudonimizado')

    for name, trace in grouper_3:
        for index in range(len(trace['Actividad'])):

            if trace['Actividad'].values[index] == attrib_name:
                if trace['Tipo intervención'].values[index] == 'PROGRAMADA': 
                    trace['Actividad'].values[index] = "Cirugia Programada"
                elif trace['Tipo intervención'].values[index] == 'URXENTE': 
                    trace['Actividad'].values[index] = "Cirugia Urgente"
                                
        traces.append(trace)
    traces_fix = pd.concat(traces)
    #renombre cirugia URXENTE
    traces_fix = traces_fix.rename(columns={"Cirugia PROGRAMADA": "Cirugia Programada", "Cirugia URXENTE": "Cirugia Urgente"})
    #eliminar actividad cita normal
    return traces_fix.drop(traces_fix.loc[traces_fix['Actividad'] == attrib_name].index)

surgery_colon = surgery_activities(colon, 'Cirugia')

In [7]:
surgery_colon['Actividad'].unique().tolist()

NameError: name 'surgery_colon' is not defined

In [8]:
groupon = colon.groupby('NASI seudonimizado')

for name, trace in groupon:
    print(name)
    print(trace['Actividad'].values.tolist())
    print('\n')

0067638e545bc471e1f89de1738c1d5c
['Cita en Medicina interna', 'Biopsia diagnóstica', 'Ingreso hospitalario', 'Cirugia', 'Alta Hospitalizacion', 'Visita médico familia', 'Cita en Oncología', 'Cita en Cirugía General', 'Visita médico familia', 'Cita en Oncología', 'Cita en Oncología', 'Cita en Cirugía General']


010b7970230a0be9031b74e8aff8df0f
['Visita médico familia', 'Biopsia diagnóstica', 'Visita médico familia', 'Alta Hospitalizacion', 'Defunción']


0129d3e82128fa38a023dc2b8d2bf445
['Visita médico familia', 'Biopsia diagnóstica', 'Cita en Cirugía General', 'Ingreso hospitalario', 'Cirugia', 'Alta Hospitalizacion', 'Visita médico familia', 'Visita médico familia', 'Cita en Oncología', 'Visita médico familia', 'Cita en Cirugía General', 'Visita médico familia', 'Visita médico familia', 'Cita en Cirugía General', 'Cita en Oncología', 'Cita en Oncología', 'Cita en Cirugía General', 'Visita médico familia', 'Visita médico familia', 'Cita en Oncología', 'Visita médico familia', 'Visita 

In [ ]:

def create_initial_activity(df):
    lista = ['Cita en Medicina interna', 'Cita en Oncología', 'Cita en Cirugía General', 'Cita en Digestivo']
    # Agrupar por el identificador de traza
    groupon = df.groupby('NASI seudonimizado')

    traces = []
    # Iterar sobre cada grupo
    for name, trace in groupon:

        # Crear una máscara booleana para identificar las filas donde 'Actividad' es 'biopsia'
        mask_biopsia = trace['Actividad'] == 'Biopsia diagnóstica'
        # Crear una máscara booleana para identificar las filas donde 'Actividad' está en la lista
        mask_lista = trace['Actividad'].isin(lista)
        # Crear una máscara booleana que marque las filas que deben eliminarse
        mask_eliminar = mask_biopsia.cumsum() == 0
        # Aplicar la máscara para eliminar las filas anteriores a 'biopsia' excepto las que están en la lista
        trace = trace[~(mask_eliminar & ~mask_lista)]
        trace = trace.reset_index(drop=True)

        primer_indice = trace.index.to_list()[0]
        print(primer_indice)
        biopsia_indice = trace[trace['Actividad'] == 'Biopsia diagnóstica'].index.min()
        print(biopsia_indice)

        if (biopsia_indice - primer_indice) > 1:
            print('resta', biopsia_indice)
            trace = trace.iloc[biopsia_indice-1:]
        traces.append(trace)

    return pd.concat(traces)           

initial_fix = create_initial_activity(colon)  

In [63]:
import numpy as np
# Función para convertir a float si es necesario
def convertir_a_float(valor):
    if isinstance(valor, np.int64):
        return int(valor)
    else:
        return valor  # Si ya es float, lo dejamos como está
    

def conditions_surgery_quimio(cirugia_index, quimioterapia_index):
    # Comprobar las condiciones posibles
    if np.isnan(cirugia_index) and not np.isnan(quimioterapia_index):
        print('El primer valor es nulo y el segundo no')
        indice_cirugia_quimioterapia = quimioterapia_index
    elif not np.isnan(cirugia_index) and np.isnan(quimioterapia_index):
        print('El primer valor no es nulo y el segundo sí')
        indice_cirugia_quimioterapia = cirugia_index
    elif not np.isnan(cirugia_index) and not np.isnan(quimioterapia_index):
        print('Ambos valores no son nulos')
        # Mantener la actividad que ocurra primero
        indice_cirugia_quimioterapia = min(cirugia_index, quimioterapia_index)
    elif np.isnan(cirugia_index) and np.isnan(quimioterapia_index):
        print('Ambos valores son nulos')
        indice_cirugia_quimioterapia = len(trace)
    
    return indice_cirugia_quimioterapia
    


In [78]:
def appointment_post_biopsia(df):
    # Agrupar por el identificador de traza
    groupon = df.groupby('NASI seudonimizado')

    traces = []
    # Iterar sobre cada grupo
    for name, trace in groupon:
        print(name)
        trace = trace.reset_index(drop=True)
        # Obtener el índice de la biopsia diagnóstica
        biopsia_indice = trace[trace['Actividad'] == 'Biopsia diagnóstica'].index.min()

        # Obtener el índice de la primera ocurrencia de 'Cirugía' y 'Quimioterapia' después de la biopsia
        cirugia_index = trace[trace.index > biopsia_indice][trace['Actividad'] == 'Cirugia'].index.min()
        quimioterapia_index = trace[trace.index > biopsia_indice][trace['Actividad'] == 'Quimioterapia'].index.min()
        cirugia_index = convertir_a_float(cirugia_index)
        quimioterapia_index = convertir_a_float(quimioterapia_index)
        print(cirugia_index)
        print(quimioterapia_index)
        #logica entre cirugia y quimio
        indice_cirugia_quimioterapia = conditions_surgery_quimio(cirugia_index, quimioterapia_index)
        print(biopsia_indice,indice_cirugia_quimioterapia)

        interna = trace[trace.index > indice_cirugia_quimioterapia][trace['Actividad'] == 'Cita en Medicina interna'].index
        Oncologia = trace[trace.index > indice_cirugia_quimioterapia][trace['Actividad'] == 'Cita en Oncología'].index
        general = trace[trace.index > indice_cirugia_quimioterapia][trace['Actividad'] == 'Cita en Cirugía General'].index
        digestivo = trace[trace.index > indice_cirugia_quimioterapia][trace['Actividad'] == 'Cita en Digestivo'].index
        medico = trace[trace.index > biopsia_indice][trace['Actividad'] == 'Visita médico familia'].index

        # cirugia_index = convertir_a_float(cirugia_index)
        # quimioterapia_index = convertir_a_float(quimioterapia_index)
        list_to_lists = []
        list_to_lists.extend(interna)
        list_to_lists.extend(Oncologia)
        list_to_lists.extend(general)
        list_to_lists.extend(digestivo)
        list_to_lists.extend(medico)
        print(list_to_lists)
    
        trace = trace.drop(list_to_lists)
        traces.append(trace)

    return pd.concat(traces)


resultados = appointment_post_biopsia(colon) 

0067638e545bc471e1f89de1738c1d5c
3
nan
El primer valor no es nulo y el segundo sí
1 3
[6, 9, 10, 7, 11, 5, 8]
010b7970230a0be9031b74e8aff8df0f
nan
nan
Ambos valores son nulos
1 27
[2]
0129d3e82128fa38a023dc2b8d2bf445
4
nan
El primer valor no es nulo y el segundo sí
1 4
[8, 14, 15, 19, 24, 26, 10, 13, 16, 22, 25, 6, 7, 9, 11, 12, 17, 18, 20, 21, 23, 27, 28]
017f4d4811a20947be153d6f8f49e1bd
8
15
Ambos valores no son nulos
1 8
[14, 16, 17, 22, 24, 25, 26, 27, 13, 20, 30, 4, 5, 10, 11, 12, 18, 28, 29, 31]
01ca2f0d170fd6ecf91984415350ce2d
11
nan
El primer valor no es nulo y el segundo sí
1 11
[35, 61, 129, 107, 5, 18, 23, 26, 27, 28, 34, 37, 38, 40, 42, 45, 46, 47, 50, 51, 55, 58, 60, 63, 67, 68, 71, 73, 78, 79, 83, 87, 88, 91, 92, 94, 97, 100, 108, 109, 111, 112, 120, 121, 122, 123, 127, 128, 130, 133, 136, 137, 138, 139, 140, 142, 143, 145, 147, 148, 150, 151, 153, 154]
03cd9c70a41eedebc2c766105812f625
nan
nan
Ambos valores son nulos
1 27
[4, 8, 9, 11]
048f62dc5a6ee6f08bf556c8c83efd2f
nan

/tmp/ipykernel_816582/2819409112.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cirugia_index = trace[trace.index > biopsia_indice][trace['Actividad'] == 'Cirugia'].index.min()
/tmp/ipykernel_816582/2819409112.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  quimioterapia_index = trace[trace.index > biopsia_indice][trace['Actividad'] == 'Quimioterapia'].index.min()
/tmp/ipykernel_816582/2819409112.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  interna = trace[trace.index > indice_cirugia_quimioterapia][trace['Actividad'] == 'Cita en Medicina interna'].index
/tmp/ipykernel_816582/2819409112.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Oncologia = trace[trace.index > indice_cirugia_quimioterapia][trace['Actividad'] == 'Cita en Oncología'].index
/tmp/ipykernel_816582/2819409112.py:26: UserWarning: Boolean Series key will be reindexed to ma

nan
nan
Ambos valores son nulos
1 27
[]
1e8640c041d510ddd1b7fcba94924d9c
11
23
Ambos valores no son nulos
1 11
[22, 24, 26, 31, 34, 39, 43, 45, 48, 50, 54, 61, 66, 71, 27, 41, 51, 57, 63, 2, 3, 4, 6, 7, 8, 9, 12, 13, 14, 15, 16, 18, 19, 20, 21, 25, 28, 29, 32, 33, 36, 37, 40, 44, 47, 52, 53, 55, 56, 58, 59, 60, 62, 64, 65, 67, 68, 69, 70]
1e93b352580e730b1d6ac80ac1a97bfd
5
nan
El primer valor no es nulo y el segundo sí
1 5
[66, 67, 15, 43, 49, 63, 23, 39, 48, 65, 7, 8, 9, 12, 13, 14, 16, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 36, 37, 38, 40, 45, 46, 47, 50, 51, 53, 57, 59, 60, 61, 62, 64]
1f2a18a452e2562390a6717959b9b166
5
nan
El primer valor no es nulo y el segundo sí
1 5
[9, 10, 15, 19, 26, 12, 16, 20, 22, 27, 33, 11, 7, 8, 13, 14, 17, 18, 21, 23, 24, 25, 32, 34]
2004a69eb176331198f29c7b49b103d5
4
nan
El primer valor no es nulo y el segundo sí
1 4
[10, 13, 18, 21, 24, 28, 45, 49, 51, 56, 59, 62, 65, 17, 19, 23, 25, 29, 42, 46, 50, 54, 55, 61, 63, 64, 5, 7, 8, 9, 

/tmp/ipykernel_816582/2819409112.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  interna = trace[trace.index > indice_cirugia_quimioterapia][trace['Actividad'] == 'Cita en Medicina interna'].index
/tmp/ipykernel_816582/2819409112.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Oncologia = trace[trace.index > indice_cirugia_quimioterapia][trace['Actividad'] == 'Cita en Oncología'].index
/tmp/ipykernel_816582/2819409112.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  general = trace[trace.index > indice_cirugia_quimioterapia][trace['Actividad'] == 'Cita en Cirugía General'].index
/tmp/ipykernel_816582/2819409112.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  digestivo = trace[trace.index > indice_cirugia_quimioterapia][trace['Actividad'] == 'Cita en Digestivo'].index
/tmp/ipykernel_816582/2819409112.py:28: UserWarning: Boolean Series key wil

[24, 26, 29, 37, 40, 43, 51, 64, 28, 38, 41, 45, 53, 61, 63, 65, 2, 4, 7, 9, 12, 14, 15, 20, 21, 23, 25, 27, 30, 31, 36, 39, 44, 46, 47, 48, 49, 50, 54, 58, 59, 60, 62]
398d11c6089ea774bc4070df752a5391
40
7
Ambos valores no son nulos
1 7
[13, 14, 17, 20, 22, 24, 27, 29, 31, 33, 34, 37, 48, 63, 77, 80, 82, 84, 85, 89, 91, 93, 95, 97, 99, 101, 104, 106, 108, 9, 28, 35, 38, 51, 81, 100, 8, 11, 21, 43, 45, 46, 47, 49, 50, 52, 53, 54, 55, 59, 61, 62, 64, 65, 66, 67, 68, 70, 71, 72, 73, 74, 76, 78, 87, 90, 96, 103, 110]
3a29641650f1138f1b57f6983659d97f
11
nan
El primer valor no es nulo y el segundo sí
1 11
[37, 15, 25, 26, 33, 36, 20, 24, 30, 31, 2, 3, 4, 6, 7, 8, 9, 13, 14, 16, 17, 19, 21, 22, 23, 29, 35]
3a71b0e89f7bca7095b4f0a0bb563d32
5
28
Ambos valores no son nulos
1 5
[9, 10, 13, 16, 19, 22, 23, 24, 26, 27, 29, 30, 32, 34, 38, 39, 41, 42, 43, 48, 49, 50, 53, 54, 55, 56, 57, 58, 61, 62, 12, 14, 17, 20, 11, 15, 18, 21, 36, 52, 7, 8, 25, 47, 64]
3b21b48aa0e98652161336c04a4b6e34
nan
15
El 

...

nan
nan
Ambos valores son nulos
1 27
[2, 8]
a69499353ecd01cf035c157010179b26
6
nan
El primer valor no es nulo y el segundo sí
1 6
[10, 12, 15, 14, 17, 11, 13, 8, 16]
a6c173b0afb4813c16a856c35ddaa6ae
nan
nan
Ambos valores son nulos
1 27
[2, 3, 5, 6, 8, 11, 12, 13]
a70a6563d1d82cd095ea19d6c9425485
11
nan
El primer valor no es nulo y el segundo sí
1 11
[5]
a71c5938c30f6bf5b46e31d8ca1be62e
5
nan
El primer valor no es nulo y el segundo sí
1 5
[9, 20, 24, 11, 14, 21, 10, 3, 7, 8, 12, 13, 15, 16, 17, 18, 19, 22, 23]
a79c46ed8f5ac089a4b515581b1126e6
6
nan
El primer valor no es nulo y el segundo sí
1 6
[3, 4]
a8115331af5613ab087f621b53c2c98f
4
nan
El primer valor no es nulo y el segundo sí
1 4
[7, 8, 15, 9, 10, 16, 11, 6, 12, 13, 14, 17]
a85a59175155f1c812524ae4eeeb49d6
14
nan
El primer valor no es nulo y el segundo sí
1 14
[69, 20, 27, 35, 46, 50, 61, 67, 24, 30, 39, 55, 63, 21, 49, 52, 68, 4, 5, 10, 11, 12, 16, 17, 19, 22, 23, 25, 26, 28, 29, 31, 32, 33, 34, 36, 37, 38, 40, 44, 45, 47, 48, 51

/tmp/ipykernel_816582/2819409112.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cirugia_index = trace[trace.index > biopsia_indice][trace['Actividad'] == 'Cirugia'].index.min()
/tmp/ipykernel_816582/2819409112.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  quimioterapia_index = trace[trace.index > biopsia_indice][trace['Actividad'] == 'Quimioterapia'].index.min()
/tmp/ipykernel_816582/2819409112.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  interna = trace[trace.index > indice_cirugia_quimioterapia][trace['Actividad'] == 'Cita en Medicina interna'].index
/tmp/ipykernel_816582/2819409112.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Oncologia = trace[trace.index > indice_cirugia_quimioterapia][trace['Actividad'] == 'Cita en Oncología'].index
/tmp/ipykernel_816582/2819409112.py:26: UserWarning: Boolean Series key will be reindexed to ma

[42, 45, 48, 53, 60, 61, 66, 30, 32, 3, 13, 14, 19, 23, 27, 28, 31, 33, 34, 35, 37, 41, 43, 44, 46, 47, 49, 50, 51, 52, 54, 55, 59, 63, 64, 65, 67, 68, 69, 70, 71, 72, 73, 75, 76, 77, 78, 79, 80, 81, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105]
c4243cd1c6e9bc298d3070fe91cebb88
5
nan
El primer valor no es nulo y el segundo sí
1 5
[12, 13, 2, 8, 9, 10, 11, 14]
c4ffdd93ba0d3446515dffe2937abd2a
2
nan
El primer valor no es nulo y el segundo sí
1 2
[8, 19, 32, 52, 62, 72, 79, 88, 17, 38, 61, 73, 78, 5, 10, 11, 12, 13, 14, 15, 16, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 55, 57, 58, 59, 60, 63, 64, 65, 66, 67, 68, 69, 70, 71, 74, 75, 76, 77, 80, 81, 82, 83, 84, 85, 86, 87, 89, 90, 91]
c597125af5ba3e8e766a0319c147c67e
5
16
Ambos valores no son nulos
1 5
[18, 15, 17, 19, 23, 25, 28, 29, 30, 31, 58, 70, 71, 75, 78, 14, 33, 56, 73, 6, 8, 9, 21, 24, 27, 32, 34, 35

/tmp/ipykernel_816582/2819409112.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cirugia_index = trace[trace.index > biopsia_indice][trace['Actividad'] == 'Cirugia'].index.min()
/tmp/ipykernel_816582/2819409112.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  quimioterapia_index = trace[trace.index > biopsia_indice][trace['Actividad'] == 'Quimioterapia'].index.min()
/tmp/ipykernel_816582/2819409112.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  interna = trace[trace.index > indice_cirugia_quimioterapia][trace['Actividad'] == 'Cita en Medicina interna'].index
/tmp/ipykernel_816582/2819409112.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Oncologia = trace[trace.index > indice_cirugia_quimioterapia][trace['Actividad'] == 'Cita en Oncología'].index
/tmp/ipykernel_816582/2819409112.py:26: UserWarning: Boolean Series key will be reindexed to ma

6
11
Ambos valores no son nulos
1 6
[10, 14, 18, 19, 20, 24, 25, 27, 28, 12, 21, 23, 26, 16, 3, 4, 8, 9, 22]
ea6d285ac76a4725787b2a80f68e5398
5
nan
El primer valor no es nulo y el segundo sí
1 5
[18, 9, 11, 13, 19, 20, 10, 17, 22, 7, 8, 21, 23, 24]
ea6ee756c7b3f250397c6887a2c516f5
3
16
Ambos valores no son nulos
1 3
[7, 8, 15, 19, 22, 24, 27, 29, 30, 34, 40, 44, 56, 60, 63, 11, 23, 32, 50, 55, 59, 65, 5, 6, 9, 10, 12, 13, 14, 18, 31, 33, 35, 36, 37, 38, 39, 41, 42, 45, 46, 47, 49, 51, 53, 57, 58, 61, 62, 64]
ea99c63cf0ad9b40fd8b600c7d5dc907
7
16
Ambos valores no son nulos
1 7
[12, 14, 19, 22, 26, 27, 31, 33, 38, 43, 46, 54, 58, 61, 63, 64, 67, 69, 71, 72, 74, 76, 77, 79, 82, 83, 84, 87, 88, 90, 91, 94, 96, 15, 39, 53, 57, 18, 85, 2, 4, 5, 9, 10, 11, 13, 17, 21, 24, 29, 32, 34, 36, 40, 44, 49, 51, 52, 55, 56, 59, 62, 66, 73]
eb40b8751742f4a54fb61a2fece84819
nan
nan
Ambos valores son nulos
1 27
[7, 10, 11, 14, 16, 17, 18, 20, 21, 22, 23, 27, 29]
ec34c3ebefd069484d676ed785061981
5
nan
El 

In [128]:
resultados_cirugia = resultados[resultados['Actividad'] == 'Cirugia']
grouponi = resultados_2.groupby('NASI seudonimizado')

for name, trace in grouponi:
    print(name)
    print(trace['Actividad'].values.tolist())
    print('\n')

0067638e545bc471e1f89de1738c1d5c
['Cita en Medicina interna', 'Biopsia diagnóstica', 'Ingreso hospitalario', 'Cirugia', 'Alta Hospitalizacion']


010b7970230a0be9031b74e8aff8df0f
['Visita médico familia', 'Biopsia diagnóstica', 'Alta Hospitalizacion', 'Defunción']


0129d3e82128fa38a023dc2b8d2bf445
['Visita médico familia', 'Biopsia diagnóstica', 'Cita en Cirugía General', 'Ingreso hospitalario', 'Cirugia', 'Alta Hospitalizacion']


017f4d4811a20947be153d6f8f49e1bd
['Cita en Cirugía General', 'Biopsia diagnóstica', 'Cita en Cirugía General', 'Entrada Lista de Espera', 'Ingreso hospitalario', 'Salida Lista de Espera', 'Cirugia', 'Alta Hospitalizacion', 'Quimioterapia']


01ca2f0d170fd6ecf91984415350ce2d
['Visita médico familia', 'Biopsia diagnóstica', 'Cita en Digestivo', 'Dispensación medicamento ATC', 'Cita en Cirugía General', 'Asistencia a punto de Atención Continuada', 'Asistencia a punto de Atención Continuada', 'Asistencia a punto de Atención Continuada', 'Asistencia a punto de A

In [126]:
def only_first_quimio(df):
    groupon = df.groupby('NASI seudonimizado')
    traces = []
    # Iterar sobre cada grupo
    for name, trace in groupon:
        
        trace = trace.reset_index(drop=True)
        indices_quimioterapia = trace.loc[trace['Actividad'] == 'Quimioterapia'].index
        if not indices_quimioterapia.empty:
            print(name)
            delete_index = indices_quimioterapia[1:]
            print(delete_index)
            trace = trace.drop(delete_index)
        traces.append(trace)
    return pd.concat(traces)

resultados_2 = only_first_quimio(resultados)


017f4d4811a20947be153d6f8f49e1bd
Index([9, 10, 11], dtype='int64')
083839c3042f1913b8f60cd80c34a1b8
Index([15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 31, 32, 33], dtype='int64')
0bcad0772ee96b94a85f3c08edf793cd
Index([8, 9], dtype='int64')
1097d97c486d37ec5ea5817b62aec1a0
Index([8], dtype='int64')
12f43d356c0cccc956f37b5369434cb4
Index([21, 22, 23, 24], dtype='int64')
16d528316dea844e421d009d296a15fd
Index([20, 21, 22, 23, 24, 25, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37], dtype='int64')
1700075f358a50f61f102c50d1259f83
Index([8, 9, 11, 13, 14, 16, 17], dtype='int64')
17657edfb2364b5bbbcc0879a61e01bf
Index([8, 9, 10, 11, 12, 13, 14, 15], dtype='int64')
17cdb9b6892753f57ded34e3fd050789
Index([11], dtype='int64')
1872b9035e4e201485446674c7503b4c
Index([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
       23, 24, 25, 26, 27, 28, 29],
      dtype='int64')
18e507b82c718455e73a63b0466f0909
Index([90, 95, 97, 103], dtype='int64')
1d9461782edde661f3ce05ded8f6cf69
I

In [52]:
def cita_extracion(df):
    # Filtrar el DataFrame para obtener solo las filas donde la actividad es 'Cita' o 'extraccion'
    df_filtered = df[df['Actividad'].isin(['Cita', 'extraccion'])]

    # Contar cuántas veces aparece cada valor de fecha asociada para cada ID
    counts = df_filtered.groupby(['NASI seudonimizado', 'Actividad', 'FECHA']).size().reset_index(name='counts')

    # Filtrar los resultados para encontrar los ID donde la fecha asociada a 'Cita' es igual a la fecha asociada a 'extraccion'
    matching_ids = counts[counts['Actividad'] == 'Data cita'].merge(counts[counts['Actividad'] == 'DATA EXTRACCIÓN'], on='NASI seudonimizado', suffixes=('_Cita', '_extraccion'))
    matching_ids = matching_ids[matching_ids['FECHA_Cita'] == matching_ids['FECHA_extraccion']]

    # Mostrar los ID resultantes
    print(matching_ids['NASI seudonimizado'].unique())


cita_extracion(colon)


[]


In [ ]:
def tc_abdominal(df):
    return df[df['Actividade CAPNOR'].str.contains('TC') & df['Actividade CAPNOR'].str.contains('ABDOM')]['Actividade CAPNOR'].unique().tolist()


lista_TCAbdominal = tc_abdominal(colon)
lista_TCAbdominal

In [ ]:
colon['Actividad'].unique().tolist()

In [ ]:
def waiting_list_duration(df, enter_list, exit_list):
    traces = []
    df['Duracion Lista Espera'] = 0
    grouper_3 = df.groupby('NASI seudonimizado')
    #pasar ambas columnas de fecha a datatime
    df[enter_list] = pd.to_datetime(df[enter_list], format= '%d-%m-%Y %H:%M:%S')
    df[exit_list] = pd.to_datetime(df[exit_list], format= '%d-%m-%Y %H:%M:%S')

    for name, trace in grouper_3:
        for index in range(len(trace['Actividad'])):

            if trace['Actividad'].values[index] == enter_list: #! trazas con lista de espera de entrada
                trace['Duracion Lista Espera'] = (trace[exit_list] - trace[enter_list]).dt.days
                trace['Duracion Lista Espera'] = trace['Duracion Lista Espera'].fillna(0)
                trace['Duracion Lista Espera'] = trace['Duracion Lista Espera'].astype(int) 
        
        traces.append(trace)
    traces_fix = pd.concat(traces)
    #eliminar actividades lista espera
    traces_fix.drop(traces_fix.loc[traces_fix['Actividad'] == enter_list].index)
    traces_fix.drop(traces_fix.loc[traces_fix['Actividad'] == exit_list].index)
    return traces_fix




colon_hardcodeado_2 = waiting_list_duration(colon, 'Data inclusión', 'Data remate')

In [ ]:
colon_hardcodeado_2[['NASI seudonimizado', 'FECHA', 'FECHA_FIN','Actividad', 'Duracion Lista Espera']].head(10)
colon_hardcodeado_2['Duracion Lista Espera'].unique()

In [ ]:
#! Añadir actividad cita con apellido del tipo de Cita y dejar unicamente los relacionados con COLONOSCOPIA u otros importantes
#'GNA','Actividade CAPNOR', 'Código Prestación', 'Prestación','Tipo vía rápida',
colon_citas = colon.query("Actividad == 'Data prescrición'")
capnor = colon_citas.groupby('Actividade CAPNOR')['NASI seudonimizado'].nunique().to_dict()
gna = colon_citas.groupby('GNA')['NASI seudonimizado'].nunique().to_dict()
# Ordenar el diccionario de mayor a menor por sus valores
capnor_ordenado = dict(sorted(capnor.items(), key=lambda item: item[1], reverse=True))
gna_ordenado = dict(sorted(gna.items(), key=lambda item: item[1], reverse=True))

for i, j in zip(capnor_ordenado.keys(),capnor_ordenado.values()):
    print(f'{i} -> {j}')

In [ ]:
#Quedarme con las actividades de tipo cita si es de tipo colonoscopia - Separar entre colonoscopia normal y VDR
#PASOS
#   1. Colocar a actividad Cita el apellido de su campo CAPNOR
#   2. Solo mantener aquellas que el apellido sea Colonoscopia o VDR u otras importantes
#   3. Con los datos en bruto, sacar ids traza que tienen colonoscopia y si son normal o VDR y colocar posteriormente como atributos

def surname_Appointments(df, attrib_name):
    traces = []
    ids_colonoscopy = []
    df['VDR'] = False
    df['Colonoscopia'] = False
    lista = ['COLONOSCOPIA', 'COLONOSCOPIA VIA RAPIDA']
    grouper_3 = df.groupby('NASI seudonimizado')

    for name, trace in grouper_3:
        for index in range(len(trace['Actividad'])):

            if trace['Actividad'].values[index] == attrib_name:
                if trace['Actividade CAPNOR'].values[index] in lista: 
                    trace['Actividad'].values[index] = 'Cita ' + trace['Actividade CAPNOR'].values[index]
                    trace['Colonoscopia'] = True
                    if trace['Tipo vía rápida'].values[index] == "VIA RAPIDA COLON              ":
                        trace['VDR'] = True

                    
        traces.append(trace)
    traces_fix = pd.concat(traces)
    #eliminar actividad cita normal
    return traces_fix.drop(traces_fix.loc[traces_fix['Actividad'] == attrib_name].index)


colon_hardcodeado = surname_Appointments(colon, 'Data prescrición')

In [ ]:
colon_hardcodeado['Actividad'].unique()
colon_hardcodeado['Colonoscopia'].unique()

In [ ]:
colon_hardcodeado['NASI seudonimizado'].nunique()

In [ ]:
colon_hardcodeado['VDR'].unique()

In [ ]:
colon.groupby('Tipo vía rápida')['NASI seudonimizado'].nunique()

In [ ]:
colon_hardcodeado.groupby('VDR')['NASI seudonimizado'].nunique()

In [ ]:
def no_colonoscopy(df):
    filtro_sin = ~df['Actividade CAPNOR'].isin(['COLONOSCOPIA', 'COLONOSCOPIA VIA RAPIDA', 'COLONOSCOPIA CON BIOPSIA'])
    filtro_con = df['Actividade CAPNOR'].isin(['COLONOSCOPIA', 'COLONOSCOPIA VIA RAPIDA', 'COLONOSCOPIA CON BIOPSIA'])
    # print(df.shape)
    # print(df['NASI seudonimizado'].nunique())
    colon_colonoscopia = df[filtro_con]
    # print(colon_colonoscopia.shape)
    # print(colon_colonoscopia['NASI seudonimizado'].nunique())
    lista_validos = colon_colonoscopia['NASI seudonimizado'].unique().tolist()

    colon_sin_colonoscopia = df[filtro_sin]
    # print(colon_sin_colonoscopia.shape)
    # print(colon_sin_colonoscopia['NASI seudonimizado'].nunique())
    return lista_validos

In [ ]:
lista  = no_colonoscopy(colon)
print(lista)
coloner = colon[colon['NASI seudonimizado'].isin(lista)]

In [ ]:
coloner['NASI seudonimizado'].nunique()

In [ ]:
colon['Actividad'].unique()

In [ ]:
def filter_dataApa(trace, value):
    trace['FECHA'] = pd.to_datetime(trace['FECHA'], format = '%d-%m-%Y %H:%M:%S')
    # Fecha límite para el filtrado, también como datetime
    limit_date = pd.to_datetime(value, format= '%d-%m-%Y %H:%M:%S')

    limit_date_minus_10s = limit_date - pd.Timedelta(seconds=10)
    trace_filtered = trace[trace['FECHA'] >= limit_date_minus_10s] 
    print(value)
    print(trace_filtered['Actividad'].unique())
    print('\n')
    return trace_filtered


def crear_dead_bolean(trace):
    trace['Defuncion'] = True
    return trace


def modify_dead_date(trace):
    max_date = trace['FECHA'].max()
    new_date = max_date + timedelta(days=30)
    new_date = pd.to_datetime(new_date, format= '%d-%m-%Y %H:%M:%S')
    # Cambiar las fechas asociadas a 'Defunción'
    trace.loc[trace['Actividad'] == 'Defunción', ['FECHA', 'FECHA_FIN']] = new_date
    return trace

In [ ]:
#recorrer traza a traza
# y para cada traza: mirar la fecha de extracción de ese paciente
#                  -> filtrar filas de ese trace superiores a esa fecha (DATA_EXTRACT)

def filter_extraction(df):

    df['FECHA'] = pd.to_datetime(df['FECHA'], format= '%d-%m-%Y %H:%M:%S')
    df['FECHA_FIN'] = pd.to_datetime(df['FECHA_FIN'], format= '%d-%m-%Y %H:%M:%S')
    grouper = df.groupby('NASI seudonimizado')

    dates, traces = [], []
    dates_dict = {}
    for name, trace in grouper:
        trace['Defuncion'] = False
        if 'Prueba laboratorio (APA)' in trace['Actividad'].unique().tolist():
            print(name)
            df = trace.query(" Actividad == 'Prueba laboratorio (APA)'")
            value = df['FECHA'].values[0]
            dates.append(value)
            dates_dict[name] = value
            
            #!boleano defunciones
            if 'Defunción' in trace['Actividad'].unique().tolist():
                trace = crear_dead_bolean(trace)
                trace = modify_dead_date(trace)
                
            print('FILTRADO!!!\n')

            traces.append(filter_dataApa(trace, value))

    return pd.concat(traces)

colon_filtered = filter_extraction(colon)

In [ ]:
# def boolAttrib_to_EmptyAttrib(df, atrrib):
#     #recorro cada traza del df
#     grouper = df.groupby('NASI seudonimizado')
#     traces = []
#     for name, trace in grouper:
#         values = trace[attrib].dropna().astype(str).unique().tolist()
#         trace[attrib]  = ', '.join(values)
#         traces.append(trace)
        

#     return pd.concat(traces)
    

# attribs = ['Prioridade', 'GNA', 'Actividade CAPNOR', 'Prestación', 'Tipo vía rápida',	'Detalle tipo actividade','Tipo programación']
# for attrib in attribs:
#     colon_filtered = boolAttrib_to_EmptyAttrib(colon_filtered, attrib)


In [ ]:
colon_filtered = colon_filtered[colon_filtered['Actividad'] != 'Preescripcion consultas y pruebas']

In [ ]:
colon_filtered['Actividade CAPNOR'].unique()

In [ ]:
def filter_important_drugs(df):
    important_drugs = ['LAXANTES OSMÓTICOS','LAXANTES FORMADORES DE VOLUMEN', 'ENEMAS']
    return df[df['Subg. químico terapéutico ATC disp'].isin(important_drugs)]

In [ ]:
def drugs_info(df):
    # #! Función que conteara el numero de veces que va a buscar medicamentos, y cuanto le cuesta en total estos medicamentos
    grouper_2 = colon_filtered.groupby('NASI seudonimizado')

    colon_filtered['Nº de compras farmaceuticas'] = 0
    colon_filtered['Coste total medicamentos'] = 0
    traces = []
    for name, trace in grouper_2: # Prezo PVP nomen disp
        cont = 0 
        try:
            cont = trace['Actividad'].value_counts()['Dispensación Farmacia Oncologica']
            print(cont)
            trace['Nº de compras farmaceuticas'] = cont
            trace['Coste total medicamentos'] = trace['Prezo PVP nomen disp'].sum()
        except KeyError:
            trace['Nº de compras farmaceuticas'] = 0
            trace['Coste total medicamentos'] = 0
        traces.append(trace)
    
    return pd.concat(traces)

colon_filtered  = drugs_info(colon_filtered)

In [ ]:
id = 'edf8341be887aa3da8126840589d7cb9'
colon_filtered_prueba = colon_filtered[colon_filtered['NASI seudonimizado'] == id]
colon_filtered_prueba['Actividad'].unique()

In [ ]:
def filter_important_drugs(df):
    for group_id, group_data in df.groupby('ID'):
        if not group_data['Algun_medicamento_importante'].any():
            # Eliminar las filas donde la actividad sea 'Dispensación medicamento ATC'
            df = df.drop(group_data[group_data['Actividad'] == 'Dispensación medicamento ATC'].index)
    return df


dfss = filter_important_drugs(colon_filtered)


In [ ]:
colon_filtered[['NASI seudonimizado', 'FECHA', 'FECHA_FIN','Actividad', 'Actividade CAPNOR','Subg. químico terapéutico ATC disp', 'Nº de compras farmaceuticas', 'Coste total medicamentos']]

In [ ]:
iterator = colon_filtered.groupby('NASI seudonimizado')

only_traces = []
for name, trace in iterator:
    activ = trace['Actividad'].unique().tolist()
    if len(activ) == 1:
        only_traces.append(name)

print(only_traces)

In [ ]:
colon_filtered['Defuncion'].value_counts()

In [ ]:
colon_filtered['FECHA'].max()

In [ ]:
colon_filtered

In [ ]:
# Eliminar '\n' y '\r' en valores y cabeceras
#colon_filtered = colon_filtered.replace({r'\r': '', r'\n': ''}, regex=True)

In [ ]:
colon_filtered.to_csv(path_gen+"Data/processed_data/CancerColon_basico_filtered"+str(version_out)+".csv", date_format="%d-%m-%Y %H:%M:%S", index=False,  encoding="UTF-8", sep = ';')

In [ ]:
no_colonoscopy(colon_filtered)